# Tutorial on running Tal's model

In [2]:
import pickle
import os

from progress_patients import * #This one is our file open it to see the functions as they are constructed

### Data transformation

In this section we transform the data as used by Tal's model to a 3 dimensional array required for this model

In [3]:
# Set seed
np.random.seed(seed=2487523)

# Hyperparameter setup.
with open(os.path.join("data", "NY_network_1.pickle"), "rb") as file:
    network = pickle.load(file)

In [4]:
wards = []
patients = []
days = network["Days"]

for t in range(len(days)):
    # Get temporary ward
    temp_ward = network["Days"][t]["Wards"]
    temp_ward["day"] = t
    wards.append(temp_ward)

    # Get temporary patient
    temp_patient = network["Days"][t]["Patients"]
    temp_patient["day"] = t
    patients.append(temp_patient)

# Get the data frame of wards and of patients
wards_df = pd.concat(wards)
patients_df = pd.concat(patients)

# Change consecutive numbers to positions
patients_df.MRN = patients_df.MRN - 1

Create the total patients per ward variable

In [5]:
# Create unique list of ward days. Move then to pandas and join the current observed values (counts)
# finally, set size to 0 if no one was there
ward_day = np.array([(x, y) for x in wards_df.ward.unique() for y in wards_df.day.unique()])
ward_day = pd.DataFrame(data=ward_day, columns=["ward", "day"])

# FIXME be careful with merging int and float here
wards = ward_day.merge(wards_df, on=["ward", "day"], how='left')
wards = wards[["day", "Size", "ward"]]
wards.loc[wards.ward == np.inf, "Size"] = np.inf
wards["Size"] = wards["Size"].fillna(0)

total_patients_per_ward = wards.to_numpy(copy=True)

Create the wards to which each patient belongs

In [6]:
# Create the wards matrix stating to which person belongs
num_patients = len(patients_df.MRN.unique())
num_days = np.max(patients_df.day.unique())
wards = np.inf * np.ones(shape=(num_days, num_patients))

for t in range(num_days):
    sub_df = patients_df[patients_df.day == t]
    wards[t, sub_df.MRN] = sub_df.ward

Create the new arrival matrix

In [7]:
# Create the new arrivals matrix
new_arrivals = np.zeros(shape=(num_days, num_patients))
unique_patients = patients_df.MRN.unique()

previous_mrn = pd.Series()
for day in range(num_days):
    # Get day's dataframe
    sub_df = patients_df[patients_df.day == day]
    new_pats = ~sub_df.MRN.isin(previous_mrn)  # Patients not in previous day
    previous_mrn = sub_df.MRN.copy(deep=True)
    new_arrivals[day, sub_df.MRN[new_pats]] = 1

Set the parameters to test

In [8]:
#PARAMETERS
#----------------------------------------------------------------
#Create the parameters
parameters = numba.typed.Dict()  
parameters["alpha"] = np.linspace(0.5, 0.7, 25)
parameters["beta"] = np.linspace(0.5, 0.7, 25)
parameters["gamma"] = np.linspace(0.5, 0.7, 25)

Get random initial values

In [9]:
#RUN MODEL
initial_colonized = np.random.binomial(n=1,p=0.005,size=sum(new_arrivals[0,] > 0)) == 1
initial_colonized = np.concatenate([initial_colonized, np.zeros(new_arrivals.shape[1] - sum(new_arrivals[0,] > 0))])
initial_colonized = initial_colonized[:,np.newaxis]
weights = np.tile(1, new_arrivals.shape)

Compile the model by running it in a small sample

In [10]:
#Here we compile the discrete model. You only need to do it once and probably will change this
#to a faster version next time
model_simulation = simulate_discrete_model(initial_colonized, wards[1:2,:], total_patients_per_ward[1:2,:], parameters, 1,
                                           new_arrivals[1:2,:], weights[1:2,:])

### Running model
Here we show how long does it take the model to run when conducting  different trajectories at the same time 

First we run the model one time to get an idea of how long it will take to run 100 times

In [11]:
%timeit -n1 simulate_discrete_model(initial_colonized, wards, total_patients_per_ward, parameters, 1, new_arrivals, weights)

SystemError: _PyEval_EvalFrameDefault returned a result with an error set

Then we run it 50 times: 

In [ ]:
%timeit -n1 simulate_discrete_model(initial_colonized, wards, total_patients_per_ward, parameters, 50, new_arrivals, weights)